## REKOMENDASI FILM


In [1]:
# Import library
import numpy as np
import pandas as pd
import seaborn as sns
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# Load dataset
movies = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')

### Exploratory Data Analysis

In [3]:
# Menampilkan data movies
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [4]:
# Menampilkan data credit
credit

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [5]:
# Menggabungkan data credit dan movies berdasarkan title
movies = movies.merge(credit, on='title')
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,9367,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,72766,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"":

In [6]:
# Mengecek informasi dataset
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

Setelah data movies dan credits digabung berdasarkan kolom title, diperoleh dataset yang berisi 4.809 baris dan 23 kolom. Kolom-kolom tersebut yaitu: 'budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'movie_id', 'cast', dan 'crew'.

In [7]:
# Mengecek dekripsi kolom numerik
movies.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count,movie_id
count,4.809000e+03,4809.000000,4809.000000,4.809000e+03,4807.000000,4809.000000,4809.000000,4809.000000
mean,2.902780e+07,57120.571429,21.491664,8.227511e+07,106.882255,6.092514,690.331670,57120.571429
std,4.070473e+07,88653.369849,31.803366,1.628379e+08,22.602535,1.193989,1234.187111,88653.369849
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,5.000000
25%,7.800000e+05,9012.000000,4.667230,0.000000e+00,94.000000,5.600000,54.000000,9012.000000
50%,1.500000e+07,14624.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000,14624.000000
75%,4.000000e+07,58595.000000,28.350529,9.291317e+07,118.000000,6.800000,737.000000,58595.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000,459488.000000


In [8]:
# Mengecek nilai dalam kolom kategorikal
categorical_columns = ['genres', 'homepage', 'keywords', 'original_language', 'original_title', 
                       'overview', 'production_companies', 'production_countries', 'release_date', 
                       'spoken_languages', 'status', 'tagline', 'title']

for categorical in categorical_columns:
    print(f"Value counts for {categorical}:")
    print(movies[categorical].value_counts())
    print("\n")  


Value counts for genres:
genres
[{"id": 18, "name": "Drama"}]                                                                                                                                             372
[{"id": 35, "name": "Comedy"}]                                                                                                                                            282
[{"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}]                                                                                                           164
[{"id": 35, "name": "Comedy"}, {"id": 10749, "name": "Romance"}]                                                                                                          144
[{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}]                                                                                                               142
                                                                                                  

In [9]:
# Mengecek nilai nul pada kolom
movies.isna().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

Terlihat bahwa jumlah nilai null terbesar terdapat pada kolom homepage, yaitu sebanyak 3.096 nilai, disusul oleh kolom tagline dengan 844 nilai null. Kolom overview memiliki 3 nilai null, sementara kolom release_date dan runtime masing-masing memiliki 1 dan 2 nilai null.

In [10]:
# Menghapus kolom homepage dan tagline
movies = movies.drop(['homepage', 'tagline'], axis=1)

Karena jumlah nilai null pada kolom-kolom tersebut terlalu tinggi, melakukan pengisian nilai akan kurang efektif. Oleh karena itu, lebih baik kolom-kolom tersebut dihapus (drop) untuk mempermudah analisis.

In [11]:
# Mengisi nilai null
movies['overview'] = movies['overview'].fillna('')
movies['release_date'] = movies['release_date'].fillna('')
movies['runtime'] = movies['runtime'].fillna('')

In [12]:
# Mengecek kembali nilai null
movies.isnull().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
title                   0
vote_average            0
vote_count              0
movie_id                0
cast                    0
crew                    0
dtype: int64

### Preprocessing

In [13]:
# Membuat subset dari dataset movies 
movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

Di sini, saya hanya menggunakan kolom 'id', 'title', 'overview', 'genres', 'keywords', 'cast', dan 'crew' untuk membangun sistem rekomendasi film berbasis Natural Language Processing (NLP). Kolom-kolom ini menyediakan informasi penting tentang identitas film, ringkasan cerita, genre, kata kunci, pemeran, dan kru yang relevan untuk analisis NLP dalam merekomendasikan film.

In [14]:
# Mengecek data teratas pada dataset 
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [15]:
# Melihat kolom genres
movies['genres']

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4804    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4805    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4806    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4807                                                   []
4808                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4809, dtype: object

Terlihat bahwa kolom genres masih mengandung simbol-simbol seperti kurung kurawal {}, kurung siku [], tanda petik "", serta atribut id dan name. Oleh karena itu, kolom ini akan dibersihkan agar hanya menyajikan nama genre, sehingga data menjadi lebih mudah diolah.

In [16]:
#  Membuat fungsi Python, yang mengambil sebuah objek string obj berisi representasi teks dari daftar atau struktur data berbentuk JSON
def convert(obj):
    L = []  # Inisialisasi daftar kosong untuk menampung hasil
    for i in ast.literal_eval(obj): # Mengubah string 'obj' menjadi objek Python dan diiterasi per item
        i['name']   # Mengakses nilai dengan kunci 'name' dari setiap elemen (dictionary)
        L.append(i['name'])  # Menambahkan nilai dari 'name' ke dalam daftar 'L'
    return L

In [17]:
# Mengaplikasikan fungsi
movies['genres']=movies['genres'].apply(convert)

In [18]:
# Mengecek kembali kolom genres
movies['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4809, dtype: object

In [19]:
# Melihat kolom keywords
movies['keywords']

0       [{"id": 1463, "name": "culture clash"}, {"id":...
1       [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2       [{"id": 470, "name": "spy"}, {"id": 818, "name...
3       [{"id": 849, "name": "dc comics"}, {"id": 853,...
4       [{"id": 818, "name": "based on novel"}, {"id":...
                              ...                        
4804    [{"id": 5616, "name": "united states\u2013mexi...
4805                                                   []
4806    [{"id": 248, "name": "date"}, {"id": 699, "nam...
4807                                                   []
4808    [{"id": 1523, "name": "obsession"}, {"id": 224...
Name: keywords, Length: 4809, dtype: object

Sama seperti kolom genres, kolom keywords juga mengandung simbol-simbol dan atribut-atribut tambahan yang perlu diproses lebih lanjut.

In [20]:
# Mengaplikasikan fungsi convert pada kolom keywords
movies['keywords'] = movies['keywords'].apply(convert)

In [21]:
# Menampilkan kembali kolom keywords
movies['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4809, dtype: object

In [22]:
# Melihat 5 data teratas
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Didapat bahwa kolom cast dan crew masih memiliki simbol-simbol dan atribut-atribut yang harus diproses lebih lanjut

In [23]:
# Mengecek kolom cast
movies['cast']

0       [{"cast_id": 242, "character": "Jake Sully", "...
1       [{"cast_id": 4, "character": "Captain Jack Spa...
2       [{"cast_id": 1, "character": "James Bond", "cr...
3       [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4       [{"cast_id": 5, "character": "John Carter", "c...
                              ...                        
4804    [{"cast_id": 1, "character": "El Mariachi", "c...
4805    [{"cast_id": 1, "character": "Buzzy", "credit_...
4806    [{"cast_id": 8, "character": "Oliver O\u2019To...
4807    [{"cast_id": 3, "character": "Sam", "credit_id...
4808    [{"cast_id": 3, "character": "Herself", "credi...
Name: cast, Length: 4809, dtype: object

In [24]:
# Mengakses elemen pertama dari kolom cast
movies['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [25]:
def convert3(obj):
    L = []  # Inisialisasi daftar kosong untuk menyimpan nama
    counter = 0  # # Inisialisasi penghitung untuk melacak jumlah nama yang telah ditambahkan
    for i in ast.literal_eval(obj): # Mengonversi string 'obj' menjadi objek Python dan iterasi melalui setiap elemen
        if counter !=3: # Mengecek apakah penghitung kurang dari 3
            L.append(i['name']) # Menambahkan nama dari elemen saat ini ke daftar L
            counter+=1 # Menambah penghitung
        else:
            break # # Menghentikan loop jika sudah mengambil 3 nama
    return L # Mengembalikan daftar L yang berisi 3 nama

In [26]:
# Mengaplikasikan fungsi covert3 kedalam kolom cast
movies['cast'] = movies['cast'].apply(convert3)

In [27]:
# Mengecek kembali kolom cast
movies['cast']

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4809, dtype: object

In [28]:
# Mengakses elemen pertama dari kolom cast
movies['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [29]:
# Membuat fungsi director yang digunakan untuk mengambil nama sutradara 
def director(obj):
    L = [] # Inisialisasi daftar kosong untuk menyimpan nama sutradara
    for i in ast.literal_eval(obj):  # Mengonversi string 'obj' menjadi objek Python dan iterasi tiap elemen
        if i['job'] == 'Director': # Memeriksa apakah pekerjaan elemen adalah 'Director'
            L.append(i['name']) # Menambahkan nama sutradara ke dalam daftar L
            break # Menghentikan loop setelah menemukan sutradara pertama
    return L # Mengembalikan daftar L yang berisi nama sutradara

In [30]:
# Mengaplikasikan fungsi director pada kolom crew
movies['crew'] = movies['crew'].apply(director)

In [31]:
# Mengecek 5 data teratas
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [32]:
# Mengubah setiap deskripsi atau ringkasan film menjadi daftar kata-kata yang terpisah
movies['overview'] = movies.overview.apply(lambda x:x.split())

In [33]:
# Mengecek kolom overview
movies['overview']

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: overview, Length: 4809, dtype: object

Dapat dilihat bahwa ringkasan yang sebelumnya berbentuk kalimat kini telah dipecah menjadi kata-kata terpisah. Proses ini dilakukan untuk mempermudah analisis dalam pemrosesan bahasa alami (NLP), sehingga setiap kata dapat dianalisis secara individual. 

In [34]:
# Menghapus spasi dari setiap elemen dalam kolom
movies['genres'] = movies.genres.apply(lambda x : [i.replace(" " , "") for i in x])
movies['keywords'] = movies.keywords.apply(lambda x : [i.replace(" " , "") for i in x])
movies['cast'] = movies.cast.apply(lambda x : [i.replace(" " , "") for i in x])
movies['crew'] = movies.crew.apply(lambda x : [i.replace(" " , "") for i in x])

Menghilangkan spasi dalam pemodelan NLP bertujuan untuk menjaga data teks tetap rapi dan konsisten agar mudah dianalisis oleh model. Spasi biasanya dihilangkan pada saat:

Tokenization: Ketika teks dipecah menjadi kata-kata atau token, spasi digunakan sebagai pemisah. Namun, dalam beberapa kasus, spasi yang tidak diperlukan dapat dihilangkan untuk menjaga keseragaman teks.

Text Normalization: Pada tahap ini, teks dinormalkan dengan cara menghapus spasi yang berlebihan, misalnya spasi ganda atau spasi di awal dan akhir kalimat, untuk membuat teks lebih rapi dan seragam.

Text Cleaning: Menghapus spasi yang tidak diperlukan adalah bagian dari pembersihan data teks untuk memastikan kualitas data yang lebih baik.

Concatenation: Pada beberapa tugas, kata atau frasa digabungkan menjadi satu string tanpa spasi. Ini dilakukan misalnya untuk membuat ID unik atau menggabungkan berbagai fitur teks menjadi satu kesatuan.

String Matching or Search: Menghilangkan spasi juga bisa membantu dalam pencocokan string, seperti mencari teks tertentu dengan lebih akurat, karena teks yang seragam mempermudah perbandingan.

In [35]:
# Membuat kolom baru bernama tags dengan menggabungkan isi dari beberapa kolom
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

Kolom tags akan menggabungkan semua informasi penting dari beberapa kolom terkait menjadi satu. Kolom ini akan berguna dalam analisis dan pemodelan, seperti ketika akan menerapkan alogritma content-based filtering untuk sistem rekomendasi.

In [36]:
# Mmebuat data frame baru bernama df
df = movies[['id', 'title', 'tags']]
df

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [37]:
# Mengubah daftar kata dalam kolom menjadi satu string yang berisi kata-kata yang dipisahkan oleh spasi
df['tags'] = df['tags'].apply(lambda x: " ".join(x))

C:\Users\User\AppData\Local\Temp\ipykernel_9440\4017501517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: " ".join(x))


In [38]:
# Mengimport library nltk untuk proses tokenisasi
import nltk
nltk.download('punkt')  # Ini akan mengunduh tokenizer untuk kalimat dan kata


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
# Mengecek tipe data
df.dtypes

id        int64
title    object
tags     object
dtype: object

In [40]:
# Mengubah tipe data menjadi string
df['tags'] = df['tags'].astype(str)

C:\Users\User\AppData\Local\Temp\ipykernel_9440\1512374881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].astype(str)


In [41]:
# Mengecek kolom tags 
df['tags']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4804    El Mariachi just wants to play his guitar and ...
4805    A newlywed couple's honeymoon is upended by th...
4806    "Signed, Sealed, Delivered" introduces a dedic...
4807    When ambitious New York attorney Sam is sent t...
4808    Ever since the second grade when he first saw ...
Name: tags, Length: 4809, dtype: object

In [42]:
# Mengubah semua teks dalam kolom 'tags' dari DataFrame df menjadi huruf kecil (lowercase)
df['tags'] = df['tags'].str.lower()

C:\Users\User\AppData\Local\Temp\ipykernel_9440\2807575439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].str.lower()


Mengubah semua teks dalam kolom tags menjadi huruf kecil (lowercase) penting untuk menghindari masalah saat melakukan analisis lebih lanjut, seperti pencocokan string atau analisis frekuensi, di mana huruf besar dan kecil dapat menyebabkan perbedaan yang tidak diinginkan. Mengubah teks menjadi huruf kecil adalah salah satu langkah pembersihan data yang harus dilakukan dalam pemrosesan teks.

In [43]:
# Mengecek duplikasi data
df.duplicated().sum()

0

In [44]:
# Mengambil nilai dari kolom 'tags' dan menyimpannya dalam variabel train_texts
train_texts = df['tags'].values

### Text Vectorization

In [45]:
# Mengubah kumpulan dokumen teks menjadi matriks jumlah kata (bag-of-words)
cv = CountVectorizer(max_features=5000, stop_words='english')

CountVectorizer merupakan kelas yang digunakan untuk melakukan tokenisasi pada teks serta menghitung frekuensi kemunculan kata dalam kumpulan dokumen. Dalam kode ini, terdapat dua parameter yang diterapkan. Pertama, max_features=5000, yang membatasi jumlah kata unik yang diambil dari dokumen. Hanya 5000 kata yang paling sering muncul yang akan disimpan dalam matriks hasil, sehingga proses ini membantu mengurangi dimensi data dan meningkatkan efisiensi model. Kedua, stop_words='english', yang berfungsi untuk menghilangkan kata-kata umum dalam bahasa Inggris yang tidak memberikan informasi signifikan, seperti "and", "the", "is", dan sebagainya. 

In [46]:
# Mengubah kolom 'tags' dari DataFrame df menjadi matriks frekuensi
vectors = cv.fit_transform(df['tags']).toarray()
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [47]:
# Mengambil daftar nama fitur (kata) yang telah dihasilkan oleh objek CountVectorizer
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

### Stemming

In [48]:
# Mmebuat objek PorterStemmer
ps = PorterStemmer()

In [49]:
# Membuat fungsi stem untuk menerapkan proses stemming pada teks
def Stem(text):
    y=[] # Inisialisasi daftar kosong untuk menyimpan hasil stemming
    for i in text.split(): # Memisahkan teks menjadi kata-kata
        y.append(ps.stem(i)) # Menerapkan stemming pada setiap kata dan menambahkannya ke daftar y

    return  " ".join(y) # Menggabungkan kata-kata yang telah distem menjadi string dan mengembalikannya

In [50]:
# Mengaplikasikan fungsi stem
df['tags'] = df['tags'].apply(Stem)

C:\Users\User\AppData\Local\Temp\ipykernel_9440\2953828147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(Stem)


### Distance (Cosine)

In [51]:
# Menghitung similarity (kemiripan) antara berbagai objek
similarity = cosine_similarity(vectors)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [52]:
similarity[0].shape

(4809,)

### Recommend The Movie

In [53]:
# Mmebuat fungsi untuk memberikan rekomendasi film
def recommend(movie):
    try:
        movie_index = df[df['title'] == movie].index[0] # Menemukan indeks film yang diberikan
        distances = similarity[movie_index] # Mengambil nilai kemiripan untuk film tersebut
        movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6] # Mengurutkan dan memilih 5 film teratas

        recommended_movies = []
        for i in movies_list:
            recommended_movies.append(df.iloc[i[0]].title)
        return recommended_movies # Menampilkan judul film yang direkomendasikan
        
    except IndexError:
        return ["Film tidak ditemukan. Pastikan judul sudah benar."]
    

    

In [54]:
recommend('Cars')

['Cars 2', 'Marmaduke', 'Rango', 'The Simpsons Movie', 'Up']

In [55]:
recommend('Superman Returns')

['Superman II',
 'Superman III',
 'Superman IV: The Quest for Peace',
 'Superman',
 'Man of Steel']